In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

: 

In [ ]:
project_dir = "."
# games = pd.read_csv(f'{project_dir}/games.csv')
# plays = pd.read_csv(f'{project_dir}/plays.csv')
# players = pd.read_csv(f'{project_dir}/players.csv')
tracking_df = pd.read_csv(f'{project_dir}/tracking_week_2.csv')
player_play = pd.read_csv(f'{project_dir}/player_play.csv')

In [ ]:
player_play_data_filtered = player_play[['gameId', 'playId', 'nflId', 'causedPressure']]
tracking_data_filtered = tracking_df[['gameId', 'playId', 'nflId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']]

# Merge the datasets on 'gameId', 'playId', and 'nflId'
merged_data = pd.merge(tracking_data_filtered, player_play_data_filtered, 
                        on=['gameId', 'playId', 'nflId'], how='inner')

# Ensure 'causedPressure' is the target variable (y)
merged_data['causedPressure'] = merged_data['causedPressure'].astype(int)  # Convert to binary if not already

print("Merged dataset shape:", merged_data.shape)